In [1]:
# Import dependencies
import os
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


import pandas as pd
import numpy as np
import tensorflow as tf
import logging
from tqdm import tqdm

# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, BinaryFocalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, NUM_CLASSES, TF_SHUFFLE_SIZE, TF_BATCH_SIZE
from src.model.tensorflow_utils import load_dataset, apply_augmentation_to_dataset, oversample_minority_classes, optimize_dataset, count_dataset_size
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_loss_monitor, setup_garbage_collector, get_metrics
from src.model.tensorflow_utils import calculate_class_weights, show_class_weights, start_or_resume_training, analyze_class_distribution
from src.model.densnet.tensorflow_dense_net_121 import build_densenet121
from src.model.loss.tensorflow_no_finding_binary_crossentropy import NoFindingBinaryCrossentropy

# Input Data
initial_epoch   = 23
resume_training = True
checkpoint_path = '/Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v3_1/checkpoints/cp-0022.keras'
model_name      = "DenseNet121_v3_1"

In [2]:
train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)
test_ds  = load_dataset(f"{TF_RECORD_DATASET}/test.tfrecord", TF_BUFFER_SIZE)

I0000 00:00:1743174745.577149 16188814 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1743174745.577457 16188814 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Optimize Dataset for rare clasess
class_weights = calculate_class_weights(train_ds, NUM_CLASSES)
train_ds      = oversample_minority_classes(train_ds, class_weights)
class_weights = calculate_class_weights(train_ds, NUM_CLASSES)

In [4]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Testing
train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds  = optimize_dataset(val_ds, TF_BATCH_SIZE)

Counting samples: 108109 samples [01:17, 1390.82 samples/s] 
Counting samples: 15391 samples [00:19, 787.15 samples/s]


In [5]:
# Setup Model Deps
# Setup Loggers
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
loss_monitor      = setup_loss_monitor(MODELS_PATH, model_name, logger, val_ds, resume_training=resume_training, initial_epoch=initial_epoch)
garbage_collector = setup_garbage_collector(logger)
metrics           = get_metrics()

# Setup compile arguments
loss       = NoFindingBinaryCrossentropy(10, lambda_value=0.1, from_logits=False, label_smoothing=0.01)
reduce_lr  = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max")

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max")

2025-03-28 16:16:39 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v3_1/train_metrics.csv
2025-03-28 16:16:39 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v3_1/val_metrics.csv
2025-03-28 16:16:39 - INFO - Cleaned training metrics file, kept 74316 records before epoch 23
2025-03-28 16:16:39 - INFO - Cleaned validation metrics file, kept 22 records before epoch 23
2025-03-28 16:16:39 - INFO - Found 74316 existing training records
2025-03-28 16:16:39 - INFO - Found 22 existing validation records
2025-03-28 16:16:39 - INFO - Cleaned loss analysis metrics file, kept 345 records before epoch 23
2025-03-28 16:16:39 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v3_1/loss_analysis_metrics.csv


In [6]:
# Model Training
model          = build_densenet121(NUM_CLASSES, use_se=True)
compile_kwargs = {'optimizer': Adam(learning_rate=1e-4, clipnorm=1.0), 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    30,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, loss_monitor, garbage_collector], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-03-28 16:16:41 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v3_1/train_metrics.csv
2025-03-28 16:16:41 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v3_1/val_metrics.csv
2025-03-28 16:16:41 - INFO - Cleaned training metrics file, kept 74316 records before epoch 23
2025-03-28 16:16:41 - INFO - Cleaned validation metrics file, kept 22 records before epoch 23
2025-03-28 16:16:41 - INFO - Found 74316 existing training records
2025-03-28 16:16:41 - INFO - Found 22 existing validation records
2025-03-28 16:16:41 - INFO - Cleaned loss analysis metrics file, kept 345 records before epoch 23
2025-03-28 16:16:41 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v3_1/loss_analysis_metrics.csv


Loading full model from checkpoint: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v3_1/checkpoints/cp-0022.keras


2025-03-28 16:16:43 - INFO - 
=== Training Started ===

2025-03-28 16:16:43 - INFO - Batch Size: 32
2025-03-28 16:16:43 - INFO - Optimizer: Adam
2025-03-28 16:16:43 - INFO - 

2025-03-28 16:16:43 - INFO - 
=== Starting Epoch 23 ===



Epoch 23/30
3378/3378 ━━━━━━━━━━━━━━━━━━━━ 0s 700ms/step - accuracy: 0.9171 - auc: 0.7996 - f1_score: 0.2672 - loss: 0.1832 - precision: 0.6616 - recall: 0.2022   

2025-03-28 16:59:07 - INFO - 
=== Epoch 23 Summary ===
2025-03-28 16:59:07 - INFO - Time: 2543.11s
2025-03-28 16:59:07 - INFO - Training   - accuracy: 0.9171 - auc: 0.8002 - f1_score: 0.2661 - loss: 0.1832 - precision: 0.6605 - recall: 0.2006 - learning_rate: 0.0001
2025-03-28 16:59:07 - INFO - Validation - accuracy: 0.9210 - auc: 0.7707 - f1_score: 0.2569 - loss: 0.2849 - precision: 0.6346 - recall: 0.1795
2025-03-28 16:59:07 - INFO - ==================================================



...

2025-03-28 23:25:37 - INFO - 
=== Epoch 30 Summary ===
2025-03-28 23:25:37 - INFO - Time: 2406.51s
2025-03-28 23:25:37 - INFO - Training   - accuracy: 0.9223 - auc: 0.8363 - f1_score: 0.3502 - loss: 0.1726 - precision: 0.7006 - recall: 0.2694 - learning_rate: 0.0000
2025-03-28 23:25:37 - INFO - Validation - accuracy: 0.9244 - auc: 0.7743 - f1_score: 0.3583 - loss: 0.2673 - precision: 0.6254 - recall: 0.2908
2025-03-28 23:25:37 - INFO - ==================================================



1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

2025-03-28 23:27:15 - INFO - 
Loss Analysis - Atelectasis
2025-03-28 23:27:15 - INFO - Confidence Distribution:
2025-03-28 23:27:15 - INFO - -- High (>0.9): 0.00%
2025-03-28 23:27:15 - INFO - -- Medium (0.6-0.9): 0.83%
2025-03-28 23:27:15 - INFO - -- Uncertain (0.4-0.6): 3.27%
2025-03-28 23:27:15 - INFO - -- Low (<0.4): 95.89%
2025-03-28 23:27:15 - INFO - Performance:
2025-03-28 23:27:15 - INFO - -- True Positives: 134
2025-03-28 23:27:15 - INFO - -- False Positives: 169
2025-03-28 23:27:15 - INFO - -- Loss Contribution: 0.3150
2025-03-28 23:27:15 - INFO - Average Confidence:
2025-03-28 23:27:15 - INFO - -- Correct Predictions: 8.30%
2025-03-28 23:27:15 - INFO - -- Incorrect Predictions: 21.28%
2025-03-28 23:27:15 - INFO - 
Loss Analysis - Cardiomegaly
2025-03-28 23:27:15 - INFO - Confidence Distribution:
2025-03-28 23:27:15 - INFO - -- High (>0.9): 0.25%
2025-03-28 23:27:15 - INFO - -- Medium (0.6-0.9): 0.96%
2025-03-28 23:27:15 - INFO - -- Uncertain (0.4-0.6): 0.97%
2025-03-28 23:27:

3378/3378 ━━━━━━━━━━━━━━━━━━━━ 2512s 714ms/step - accuracy: 0.9217 - auc: 0.8313 - f1_score: 0.3393 - loss: 0.1748 - precision: 0.7003 - recall: 0.2603 - val_accuracy: 0.9244 - val_auc: 0.7743 - val_f1_score: 0.3583 - val_loss: 0.2673 - val_precision: 0.6254 - val_recall: 0.2908 - learning_rate: 5.0000e-05


2025-03-28 23:27:23 - INFO - 
=== Training Completed! ===

2025-03-28 23:27:23 - INFO - Final Metrics: accuracy: 0.9223 - auc: 0.8363 - f1_score: 0.3502 - loss: 0.1726 - precision: 0.7006 - recall: 0.2694 - val_accuracy: 0.9244 - val_auc: 0.7743 - val_f1_score: 0.3583 - val_loss: 0.2673 - val_precision: 0.6254 - val_recall: 0.2908

